## Model Pipeline
**Data preprocessing**: Resize to 224x224, binarize, and apply n=2 morphological thinning (detect model); Resized to 224x224 with no filters (classify model)

**Model**: Tandem inference implementation of the detection and classification model from the Yu, et al [paper](https://drive.google.com/file/d/1nYl4w41CAcj8XwTEdVwcD5lVheUFIHVy/view?usp=sharing)

In [ ]:
!pip install pickle5

In [ ]:
# import libraries
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, losses, optimizers, regularizers, callbacks

import os
import time
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

import cv2
from scipy.ndimage import median_filter
from skimage.transform import resize as sk_resize
from skimage.util import img_as_ubyte
from skimage.morphology import skeletonize, thin

import helpers as helper
from keras_model_s3_wrapper import *

import boto3
import pickle5 as pickle
s3 = boto3.resource('s3')
bucket_name = 'wafer-capstone'
my_bucket = s3.Bucket(bucket_name)

In [ ]:
tf.__version__

In [ ]:
tf.config.list_physical_devices(device_type=None)

### Load the data
Dataset must have the following columns: 
- **waferMap**: defect data of wafer map where 0 = blank spot, 1 = normal die (passed the electrical test), and 2 = broken die (failed electrical test)
- **ID**: unique identification for each waferMap, separate from dataframe index

If labeled, dataset must have the following columns:
- **detectLabels**: for evaluating the detect model, where 0 = no defect, 1 = defect
- **classifyLabels**: for evaluating the classify model, where 0 = Loc, 1 = Edge-Loc, 2 = Center, 3 = Edge-Ring, 4 = Scratch, 5 = Random, 6 = Near-full, 7 = Donut, 8 = none

In [ ]:
# specify variables

# specify data to load
path = '' # S3 folder where data lives
filename = '' # data filename in S3
labeled = True

# where to save results
result_path = '' # folder in local instance to save results
result_filename = '' # filename to save the results as

# which models to run
detect_model = 'yudetect-224-thin2'
classify_model = 'yuclassify-224'

In [ ]:
# load data directly from S3 (using boto3 resource)
start = time.time()

data_key = f'{path}/{filename}.pkl'
data = pickle.loads(my_bucket.Object(data_key).get()['Body'].read())

print("Wall time: {:.2f} seconds".format(time.time() - start))
print(f"Dataset length: {len(data)}")

In [ ]:
# IF LABELED
# show failure type distribution
if labeled:
    data_defects = data[data.detectLabels == 1]
    helper.defect_distribution(data_defects, note=f'({filename})')

### Data preprocessing

In [ ]:
# resize to 224x224
start = time.time()

def resize(x):
    y = sk_resize(x, [224,224])
    new_y = img_as_ubyte(y)
    return new_y
    
data['waferMap224'] = data.waferMap.apply(lambda x: resize(x))

print("Wall time: {:.2f} seconds".format(time.time() - start))
print("Sanity checks:")
print(f'Map shape: {data.waferMap224[0].shape}')

In [ ]:
# resize to 224x224 --> binarize --> apply n=2 morphological thinning
start = time.time()

def preprocess(x):
    ret, thresh_img = cv2.threshold(x, 1, 1, cv2.THRESH_BINARY)
    thin_img = thin(thresh_img, 2)
    return thin_img
    
data['thinMap2'] = data.waferMap224.apply(lambda x: preprocess(x))
data['thinMap2'] = data.thinMap2.apply(lambda x: x.astype(np.uint8))

print("Wall time: {:.2f} seconds".format(time.time() - start))
print("Sanity checks:")
print(f'Map shape: {data.thinMap2[0].shape}')

### Detect Model

##### Detect data set-up

In [ ]:
# prepare inputs
start = time.time()

x_det = np.stack(data['thinMap2'])
x_det = tf.expand_dims(x_det, axis=3, name=None)

print("Wall time: {:.2f} seconds".format(time.time() - start))
# sanity check
# expected: TensorShape([#rows, xdim, ydim, 1])
x_det.shape

In [ ]:
# IF LABELED
# prepare labels for evaluating results
if labeled:
    y_det = np.asarray(data['detectLabels']).astype(np.uint8)

##### Load and run detect model

In [ ]:
# load saved detect model from S3
start = time.time()

detect = s3_get_keras_model(detect_model)
detect.summary()

print("Wall time: {:.2f} seconds".format(time.time() - start))

In [ ]:
# generate predictions
start = time.time()

detect_pred = detect.predict(x_det)
det_labels = np.argmax(detect_pred, axis=1).astype(np.uint8)

print("Wall time: {:.2f} seconds".format(time.time() - start))

In [ ]:
# IF LABELED
# evaluate detect model performance
if labeled:
    
    # calculate baseline accuracy
    nones = len(data[data.detectLabels == 0])
    total = len(data)
    print(f"Baseline accuracy: {nones/total*100:.2f}%")
    
    # manually compute detect model accuracy
    det_cm = confusion_matrix(y_det, det_labels)
    det_accuracy = (det_cm[0][0] + det_cm[1][1]) / len(y_det) * 100
    print(f'Detection Model Accuracy: {det_accuracy:.2f}%')
    
    # plot confusion matrix
    helper.plot_confusion_matrix(y_det, det_labels, mode='detect', normalize=True)

### Classify Model

##### Classify data set-up

In [ ]:
# keep only subset of test data
# predicted by detect model as having defects
defect_indices = [i for i in range(len(det_labels)) if det_labels[i] == 1]
defect_ids = [data.ID[i] for i in defect_indices]
defect_df = data.loc[defect_indices].reset_index(drop=True)

# sanity check:
print("Sanity checks:")
print(f'{len(defect_indices)}, {len(defect_ids)}, {defect_df.shape}')

In [ ]:
# prepare inputs
start = time.time()

x_cls = np.stack(defect_df['waferMap224'])
x_cls = tf.expand_dims(x_cls, axis=3, name=None)

print("Wall time: {:.2f} seconds".format(time.time() - start))
# sanity check
# expected: TensorShape([#rows, xdim, ydim, 1])
x_cls.shape

In [ ]:
# IF LABELED
# prepare labels for evaluating results
if labeled:
    y_cls = np.asarray(defect_df['classifyLabels']).astype(np.uint8)

##### Load and run classify model

In [ ]:
# load saved classify model from S3
start = time.time()

classify = s3_get_keras_model(classify_model)
classify.summary()

print("Wall time: {:.2f} seconds".format(time.time() - start))

In [ ]:
# generate predictions
start = time.time()

classify_pred = classify.predict(x_cls)
cls_labels = np.argmax(classify_pred, axis=1).astype(np.uint8)

print("Wall time: {:.2f} seconds".format(time.time() - start))

### Collect tandem model results
Saved predictions include 4 lists:
- IDs of defective wafers identified by detect model
- Output of detect model (softmax probabilities)
- Output of classify model (softmax probabilities)
- Labels predicted by tandem model

In [ ]:
# generate full prediction
def tandem_prediction(x):
    if x in set(defect_ids):
        i = defect_ids.index(x)
        return cls_labels[i]
    else:
        return 8

data['tandemLabels'] = data.ID.apply(lambda x: tandem_prediction(x))
tandem_pred = data['tandemLabels'].tolist()
print(len(tandem_pred))

In [ ]:
# save predictions to local instance
predictions = [defect_ids, detect_pred, classify_pred, tandem_pred]
with open(f'{result_path}/{result_filename}.pkl', "wb") as f:
    pickle.dump(predictions, f)

In [ ]:
# IF LABELED
if labeled:
    y_test = data['classifyLabels'].tolist()
    
    # manually compute overall accuracy
    tandem_cm = confusion_matrix(y_test, tandem_pred)

    tandem_num = 0
    for i in range(9):
        tandem_num += tandem_cm[i][i]

    overall_accuracy = tandem_num / len(y_test) * 100
    print(f'Overall Model Accuracy: {overall_accuracy:.2f}%') 

    # plot confusion matrix
    helper.plot_confusion_matrix(y_test, tandem_pred, mode='all', normalize=True)

### Error Analysis

In [ ]:
# IF LABELED
# plot confusion matrix counts
if labeled:
    helper.plot_confusion_matrix(y_test, tandem_pred, mode='all', normalize=False)

##### Optional visualization of misclassified wafers
Parameters:
- **true_label**: true label of the wafer
- **pred_label**: label predicted by the model
- **n**: number of samples to visualize (note: must be less than or equal to the total number in confusion matrix)

0 = Loc, 1 = Edge-Loc, 2 = Center, 3 = Edge-Ring, 4 = Scratch, 5 = Random, 6 = Near-full, 7 = Donut, 8 = none

In [ ]:
# # plot mislabeled wafers
# print('Scratch mislabled as None')
# helper.visualize_misclassified(data, y_test, tandem_pred, true_label=4, pred_label=8, n=9, 
#                         figsize=(5,5), col='waferMap', cmap='gray_r')